## Globals

In [ ]:
# PRETRAINED = ""
# FREQ_MASK_PARAM = 7
# TIME_MASK_PARAM = 7
# MASK = False
COLAB = True
SAVE_DIR = "/content/MyDrive/MyDrive/s24-idl/hw1p2/alec/output"
SAVE_THRESHOLD = 0.84

In [ ]:
if COLAB:
  from google.colab import drive
  drive.mount('/content/MyDrive/')

Mounted at /content/MyDrive/


In [ ]:
!pip install torchsummaryX wandb torchaudio librosa --quiet

import torch
import numpy as np
from torchsummaryX import summary
import torchaudio
import torchaudio.transforms as T
import librosa
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# setup kaggle data
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"alectrela","key":"42d0f6df23d52bb3b9e5a7b2cb5e1962"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

# commands to download data from kaggle
!kaggle competitions download -c 11785-hw1p2-s24

!unzip -qo /content/11785-hw1p2-s24.zip -d '/content'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
Device:  cuda
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73248 sha256=1a5dc6cd77bf028455773ec67b728f6ca9a38d4faf434b03cbe46ffacb7a8c02
  Stored in directory: /root/.cache/pip/wheels/0b/76/ca/e58f8afa83166a0e68f0d5cd2e7f99d260bdc40e35da080eee
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16
100% 3.74G/3.75G [00:15<00:00, 259MB/s]
100% 3.75G/3.75G [00:15<00:00, 256MB/s]


In [ ]:
#@title

# config = {
#     'epochs'        : 30,
#     'batch_size'    : 2048,
#     'context'       : 30,
#     'init_lr'       : 0.01,
#     'architecture'  : 'diamond',
#     'repeat'        : 0,
#     'depth'         : 8,
#     'optimizer'     : "AdamW",
#     'scheduler'     : "StepLR(optimizer, step_size=1, gamma=0.35, last_epoch=-1",
#     'activations'   : "PReLU",
#     'notes'         : "Batchnorm, He init, mask(7/7)",
#     'dropout'       : "[1/0.15, 2/0.3, 3/0.3, 1/0.15]",
#     # "pyramid step"  : "start_width, step = 2700, 360"
#     # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
# }

## Parameter Sweep

In [ ]:
config = {
    'method': "random",
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            # # a flat distribution between 0 and 0.1
            'distribution': 'uniform',
            'min': 0.008,#0.005,
            'max': 0.01,#0.05
            # start based on where other models finished after sweep 2
        },
        'mask' : {
            "values": [6]
        },
        "scheduler": {
            "values": ['lin']
        },
        'dropout': {
            'values': [0.25]
        },
        'activation': {
            "values": ['gelu']
        },
        'context': {
            'values': [35]
        },
        'batch_size': {
            'values': [2048]
        },
        'epochs': {
            'value': 60
        }
    }
}

# ======================= methods for sweep ===========================

def build_dataset(context, mask) -> tuple:

    train_data = AudioDataset("/content/11-785-s24-hw1p2", context=context, mask=mask)
    val_data = AudioDataset("/content/11-785-s24-hw1p2", context=context, partition="dev-clean")
    test_data = AudioTestDataset("/content/11-785-s24-hw1p2", context=context, partition="test-clean")

    return train_data, val_data, test_data

def build_loaders(train_data, val_data, test_data, batch_size):

    train_loader = torch.utils.data.DataLoader(
        dataset     = train_data, num_workers = 4,
        batch_size  = batch_size, pin_memory  = True,
        shuffle     = True
    )

    val_loader = torch.utils.data.DataLoader(
        dataset     = val_data, num_workers = 2,
        batch_size  = batch_size, pin_memory  = True,
        shuffle     = False
    )

    test_loader = torch.utils.data.DataLoader(
        dataset     = test_data, num_workers = 2,
        batch_size  = batch_size, pin_memory  = True,
        shuffle     = False
    )

    return train_loader, val_loader, test_loader

def build_model(input_size, dropout, activation, frame, output_size=42):

    torch.cuda.empty_cache()
    gc.collect()

    # https://stackoverflow.com/questions/75967951/pytorch-weight-initialization-problem-for-dcgan
    def init_weight(model):
        def _init_weight(module):
            if isinstance(module, torch.nn.Linear):
                torch.nn.init.kaiming_uniform_(module.weight, mode='fan_in', nonlinearity='relu')
        model.apply(_init_weight)

    # Create model and push to GPU
    # INPUT_SIZE  = (2*context + 1) * 27 # Why is this the case?
    model       = Network(input_size, dropout, activation).to(device)
    init_weight(model)

    summary(model, frame.to(device))

    print("\n==================================================\n")

    return model

def get_scheduler(optimizer, scheduler):
  if scheduler == 'exp':
    return torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.8)
  elif scheduler == 'lin':
    return torch.optim.lr_scheduler.LinearLR(optimizer)
  else:
    return torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)



## Dataset Classes


In [ ]:
def apply_freq_masking(S_dB, n_freq_mask):
    S_dB_augmented = np.copy(S_dB)
    for _ in range(n_freq_mask):
        freq_band = np.random.randint(0, S_dB.shape[0])
        S_dB_augmented[freq_band, :] = 0
    return S_dB_augmented

def apply_time_masking(S_dB, n_time_mask):
    S_dB_augmented = np.copy(S_dB)
    for _ in range(n_time_mask):
        time_band = np.random.randint(0, S_dB.shape[1])
        S_dB_augmented[:, time_band] = 0
    return S_dB_augmented

# ================================= Train/Val Set ======================================

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100", mask=0): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes
        self.mask       = mask

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = os.path.join(root + "/" + partition + "/mfcc/")
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir =  os.path.join(root + "/" + partition + "/transcript/")

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    = sorted(os.listdir(self.transcript_dir))

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(os.path.join(self.mfcc_dir + "/" + mfcc_names[i]))
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc        = (mfcc - mfcc.mean(axis=1, keepdims=True)) / (mfcc.std(axis=1, keepdims=True) + 1e-5)
        #   Load the corresponding transcript
            transcript  = np.load(os.path.join(self.transcript_dir + "/" + transcript_names[i]))[1:-1] # Remove [SOS] and [EOS] from the transcript
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2), (T2+2) before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 27 (Where T = T1 + T2 + ...)
        self.mfccs          = np.vstack(self.mfccs)

        # print("mfccs shape: ", self.mfccs.shape)

        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts    = np.concatenate(self.transcripts)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is.
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, pad_width=((context, context), (0, 0)), mode="constant") # TODO

        # print("mfccs padded shape: ", self.mfccs.shape)


        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        self.transcripts = np.array([self.phonemes.index(phenome) for phenome in self.transcripts])
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.

        # THINK ABOUT THE OFFSET & THE CONTEXT INTERACTING
        frames = self.mfccs[ind: ind + (2*self.context) + 1, :]

        if self.mask:
          # with 50% prob, mask the data
          if np.random.rand() > 0.5:
              frames = apply_time_masking(frames, n_time_mask=self.mask)

          if np.random.rand() > 0.5:
              frames = apply_freq_masking(frames, n_freq_mask=self.mask)

        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.reshape(-1) # TODO: Flatten to get 1d data
        # print(frames.shape)

        frames      = torch.FloatTensor(frames) # Convert to tensor
        phonemes    = torch.tensor(self.transcripts[ind])

        return frames, phonemes

# ================================= Train/Val Set ======================================

# ================================= Test Set ======================================

class AudioTestDataset(torch.utils.data.Dataset):

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

    def __init__(self, root, context=0, partition= "test-clean"): # Feel free to add more arguments

        self.context    = context

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = os.path.join(root + "/" + partition + "/mfcc/")

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order

        self.mfccs =  []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(os.path.join(self.mfcc_dir + "/" + mfcc_names[i]))
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc        = (mfcc - mfcc.mean(axis=1, keepdims=True)) / (mfcc.std(axis=1, keepdims=True) + 1e-5)

            self.mfccs.append(mfcc)

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2), (T2+2) before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 27 (Where T = T1 + T2 + ...)
        self.mfccs          = np.vstack(self.mfccs)


        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        self.mfccs = np.pad(self.mfccs, pad_width=((context, context), (0, 0)), mode="constant") # TODO

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers


    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind: ind + (2*self.context) + 1, :]
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.reshape(-1) # TODO: Flatten to get 1d data
        # print(frames.shape)

        frames = torch.FloatTensor(frames) # Convert to tensors

        return frames

# ================================= Test Set ======================================


# TODO: Create a dataset object using the AudioTestDataset class for the test data
# test_data = AudioTestDataset("/content/11-785-s24-hw1p2", context=35, partition="test-clean")


In [ ]:
#@title

# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

# train_loader = torch.utils.data.DataLoader(
#     dataset     = train_data,
#     num_workers = 4,
#     batch_size  = config['parameters']['batch_size']["value"],
#     pin_memory  = True,
#     shuffle     = True
# )

# val_loader = torch.utils.data.DataLoader(
#     dataset     = val_data,
#     num_workers = 2,
#     batch_size  = config['parameters']['batch_size']["value"],
#     pin_memory  = True,
#     shuffle     = False
# )

# test_loader = torch.utils.data.DataLoader(
#     dataset     = test_data,
#     num_workers = 2,
#     batch_size  = config['parameters']['batch_size']["value"],
#     pin_memory  = True,
#     shuffle     = False
# )


# print("Batch size     : ", config['parameters']['batch_size']["value"])
# print("Context        : ", config['parameters']['context']['value'][-1])
# print("Input size     : ", (2*config['parameters']['context']['value'][-1]+1)*27)
# print("Output symbols : ", len(PHONEMES))

# print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
# print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
# print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

In [ ]:
#@title

# Testing code to check if your data loaders are working
# for i, data in enumerate(train_loader):
#     frames, phoneme = data
#     print(frames.shape, phoneme.shape)
#     break


In [ ]:
#@title

# class Network(torch.nn.Module):

#     def __init__(self, input_size, output_size):

#         super(Network, self).__init__()

#         start_size = 2370
#         layer = lambda x, y: math.floor(x*(0.5**y) )

#         self.layers = []

#         self.layers.append(torch.nn.Linear(input_size, layer(start_size, i)))
#         self.layers.append(torch.nn.BatchNorm1d(layer(start_size, i)))
#         self.layers.append(torch.nn.PReLU())
#         for i in range(config["depth"]):


#             # print(input_size, layer(start_size, i))

#           for _ in range(config["repeat"] - 1):
#             self.layers.append(torch.nn.Linear(layer(start_size, i), layer(start_size, i)))
#             self.layers.append(torch.nn.BatchNorm1d(layer(start_size, i)))
#             self.layers.append(torch.nn.PReLU())
#             # print(layer(start_size, i), layer(start_size, i))

#           self.layers.append(torch.nn.Dropout( 1 / ( (i+1) * 4 ) ))

#           if i == config["depth"] - 1:
#             self.layers.append(torch.nn.Linear(layer(start_size, i), output_size))

#           else:
#             self.layers.append(torch.nn.Linear(layer(start_size, i), layer(start_size, i+1)))
#             self.layers.append(torch.nn.BatchNorm1d(layer(start_size, i+1)))
#             self.layers.append(torch.nn.PReLU())

#         self.model = torch.nn.Sequential(*self.layers)

#     def forward(self, x):

#         out = self.model(x)

#         return out

In [ ]:
class Network(torch.nn.Module):

    def __init__(self, input_size, dropout_rate, activation, output_size=42):

        super(Network, self).__init__()

        self.layers = []

        l = [ input_size, 1300, 2100, 3350, 2200, 1200, 700, 400]
        dropout = [1, 2, 3, 4]
        j = 0

        for i in range(len(l[:-1])):

          self.layers.append(torch.nn.Linear(l[i], l[i+1]))
          self.layers.append(torch.nn.BatchNorm1d(l[i+1]))

          if activation == "prelu":
              self.layers.append(torch.nn.PReLU())
          elif activation == 'silu':
              self.layers.append(torch.nn.SiLU())
          else:
              self.layers.append(torch.nn.GELU())

          if i in dropout:
            self.layers.append(torch.nn.Dropout(dropout_rate))
            j += 1

        self.layers.append(torch.nn.Linear(l[-1], output_size))

        self.model = torch.nn.Sequential(*self.layers)

    def forward(self, x):

        out = self.model(x)

        return out



In [ ]:
# train_data, val_data, test_data = build_dataset(35, 5)
# train_loader, val_loader, test_loader = build_loaders(train_data, val_data, test_data, 2048)


# # make sure frames is working
# for i, data in enumerate(train_loader):
#   frames, phoneme = data
#   print(frames.shape, phoneme.shape)
#   break

# input_size = (2*35+1)*27
# model = build_model(input_size, 0.2, "silu", frames)

# del model
# torch.cuda.empty_cache()
# gc.collect()

In [ ]:
#@title

# Create model and push to GPU
# INPUT_SIZE  = (2*40 + 1) * 27 # Why is this the case?
# model       = Network(INPUT_SIZE, len(train_data.phonemes), ).to(device)
# init_weight(model)

# Load pretrained model
# if PRETRAINED:
#   model.load_state_dict(torch.load(PRETRAINED))

# summary(model, frames.to(device))

# # setup extra & garbage collect
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr= config['init_lr'])
# # stepLR = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.35, last_epoch=-1)
# plateauLR = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, threshold=0.005, patience=2)
# scheduler = ChainedScheduler([scheduler1, scheduler2])
# https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ChainedScheduler.html

# torch.cuda.empty_cache()
# gc.collect()

In [ ]:
# https://pytorch.org/docs/stable/notes/amp_examples.html

def train(model, dataloader, optimizer, criterion, scaler):

    model.train()

    # TRAIN FOR HALF THE DATA
    train_tot = len(dataloader)

    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=train_tot, dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        # cutoff
        # if i > train_tot: break

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        with torch.autocast(device_type=device):
          ### Forward Propagation
          logits  = model(frames)

          ### Loss Calculation
          loss    = criterion(logits, phonemes)

        ### Backward Propagation
        scaler.scale(loss).backward()

        ### Gradient Descent
        scaler.step(optimizer)

        scaler.update()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= train_tot
    tacc    /= train_tot

    return tloss, tacc

def eval(model, dataloader, criterion):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(dataloader)
    vacc    /= len(dataloader)

    return vloss, vacc

In [ ]:
#@title

# run = wandb.init(
#     name    = run_name, ### Wandb creates random run names if you skip this field, we recommend you give useful names
#     reinit  = True, ### Allows reinitalizing runs when you re-run this cell
#     #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
#     #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
#     project = "hw1p2", ### Project should be created in your wandb account
#     config  = config ### Wandb Config for your run
# )

## Wandb Sweep Init

In [ ]:
# @title Default title text

### Save your model architecture as a string with str(model)
# model_arch  = str(model)

# ### Save it in a txt file
# arch_file   = open(os.path.join(SAVE_DIR + f"/{run_name}.txt"), "w")
# file_write  = arch_file.write(model_arch)
# arch_file.close()

# ### log it in your wandb run with wandb.save()
# wandb.save(os.path.join(SAVE_DIR + f'/{run_name}.txt'))

In [ ]:

wandb.login(key="58cec1d955c8a558ddb5374428c04433fd0455c6") #API Key is in your wandb account, under settings (wandb.ai/settings)
sweep = wandb.sweep(config, project="hw1p2-sweep")

def train_sweep(config=None):
  with wandb.init(config=config) as run:

    torch.cuda.empty_cache()
    gc.collect()

    params = wandb.config

    run.name = run_name = f"beat-bian-sun"

    train_data, val_data, test_data = build_dataset(params.context, params.mask)
    train_loader, val_loader, test_loader = build_loaders(train_data, val_data, test_data, params.batch_size)

    SAVE_THRESHOLD = 0.84
    # PATH="/content/MyDrive/MyDrive/s24-idl/hw1p2/alec/output/ft-dia-L9e-03-M6-D0.25-gelu-C35_E38.pt"

    # make sure frames is working
    for i, data in enumerate(train_loader):
      frames, phoneme = data
      print(frames.shape, phoneme.shape)
      break

    input_size = (2*params.context+1)*27

    model = build_model(input_size, params.dropout, params.activation, frames)
    # IF LOADING PRETRAINED
    # model.load_state_dict(torch.load(PATH))

    ### Save it in a txt file
    model_arch  = str(model)
    arch_file   = open(os.path.join(SAVE_DIR + f"/{run_name}.txt"), "w")
    file_write  = arch_file.write(model_arch)
    arch_file.close()

    optimizer = torch.optim.AdamW(model.parameters(), lr=params.learning_rate)
    criterion = torch.nn.CrossEntropyLoss()
    scheduler = get_scheduler(optimizer, params.scheduler)
    plateauLR = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.65, threshold=0.025, patience=2)

    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(params.epochs):

        print("\nEpoch {}/{}".format(epoch+1, params.epochs))

        curr_lr                 = float(optimizer.param_groups[0]['lr'])
        train_loss, train_acc   = train(model, train_loader, optimizer, criterion, scaler)

        val_loss, val_acc       = eval(model, val_loader, criterion)

        # if epoch <= 5:
          # scheduler.step()
        # elif epoch == 35:
        #   for param_group in optimizer.param_groups:
        #     param_group['lr'] *= 10
        # else:
        plateauLR.step(val_loss)

        # scheduler.step() # step after each epoch

        print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
        print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

        ### Log metrics at each epoch in your run
        # Optionally, you can log at each batch inside train/eval functions
        # (explore wandb documentation/wandb recitation)
        wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
                  'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

        ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
        if val_acc > SAVE_THRESHOLD:
            SAVE_THRESHOLD = val_acc
            torch.save(model.state_dict(), os.path.join(SAVE_DIR + f"/{run_name}_E{epoch}.pt"))

    del model
    run.finish()

### Finish your wandb run
# run.finish()
wandb.agent(sweep, train_sweep, count=3)

## Save the Model if Need Be

In [ ]:
# torch.save(model.state_dict(), os.path.join( SAVE_DIR + f"/{run_name}.pt"))

In [ ]:
ind_config = {
    'method': "random",
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            # # a flat distribution between 0 and 0.1
            'distribution': 'uniform',
            'min': 0.003,#0.005,
            'max': 0.006,#0.05
            # start based on where other models finished after sweep 2
        },
        'mask' : {
            "values": [7]
        },
        "scheduler": {
            "values": ['lin']
        },
        'dropout': {
            'values': [0.25]
        },
        'activation': {
            "values": ['silu']
        },
        'context': {
            'values': [35]
        },
        'batch_size': {
            'value': 2048
        },
        'epochs': {
            'value': 30
        }
    }
}

## Test with Model

In [ ]:

def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim=1)

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            # TODO
            for i in range(len(predicted_phonemes)):
              test_predictions.append(PHONEMES[predicted_phonemes[i]])

    return test_predictions

predictions = test(model, test_loader)

### Create CSV file with predictions
with open(f"{SAVE_DIR}/submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))